In [2]:
# Word2Vec

In [1]:
import collections
import math
import os
import errno
import random
import zipfile
import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

ImportError: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['C:\\Users\\NikolayStanchev\\Anaconda3\\lib\\site-packages\\numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

In [2]:
data_dir = 'word2vec_data/words'
data_url = 'http://mattmahoney.net/dc/text8.zip'

In [3]:
def fetch_words_data(url=data_url, words_data=data_dir):
    
    # Make the Dir if it does not exist
    os.makedirs(words_data, exist_ok=True)
    
    # Path to zip file 
    zip_path = os.path.join(words_data, "words.zip")
    
    # If the zip file isn't there, download it from the data url
    if not os.path.exists(zip_path):
        urllib.request.urlretrieve(url, zip_path)
        
    # Now that the zip file is there, get the data from it
    with zipfile.ZipFile(zip_path) as f:
        data = f.read(f.namelist()[0])
    
    # Return a list of all the words in the data source.
    return data.decode("ascii").split()

In [4]:
words = fetch_words_data()

In [5]:
len(words)

17005207

In [6]:
words[900:910]

['a',
 'form',
 'of',
 'amoralism',
 'in',
 'which',
 'individuals',
 'would',
 'unite',
 'in']

In [ ]:
for w in words[900:910]:
    print(w,end = ' ')

In [ ]:
from collections import Counter

In [ ]:
my_list = ['one','two','two']

In [ ]:
Counter(my_list)

In [ ]:
def create_counts(vocab_size = 50000):
    
    vocab = [] + Counter(words).most_common(vocab_size)
    
    vocab = np.array([word for word, _ in vocab])
    
    dictionary = {word:code for code, word in enumerate(vocab)}
    
    data = np.array([dictionary.get(word,0) for word in words])
    
    return data,vocab
    
    

In [ ]:
data , vocabulary = create_counts()

In [ ]:
data.shape

In [ ]:
vocabulary.shape

In [ ]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
    if data_index == len(data):
        buffer[:] = data[:span]
        data_index = span
    else:
        buffer.append(data[data_index])
        data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [ ]:
# Constants

In [ ]:
batch_size = 128
embedding_size = 150
skip_window = 1
num_skips = 2

In [ ]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window,valid_size,replace = False)

In [ ]:
num_sampled = 64 
learning_rate = .01
vocabulary_size = 50000

In [ ]:
tf.reset_default_graph()
train_inputs = tf.placeholder(tf.int32,shape = [None])

In [ ]:
train_labels = tf.placeholder(tf.int32,shape = [batch_size,1])

In [ ]:
valid_dataset = tf.constant(valid_examples,dtype = tf.int32)

In [ ]:
init_embeds = tf.random_uniform([vocabulary_size,embedding_size],-1,1)

In [ ]:
embeddings = tf.Variable(init_embeds)

In [2]:
embed = tf.nn.embedding_lookup(embeddings,train_inputs)

NameError: name 'tf' is not defined

In [7]:
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size,]))

NameError: name 'tf' is not defined